<a href="https://colab.research.google.com/github/Svetlana20005/Python/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%D0%9E%D0%9E%D0%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Вариант 7. Учет и управление фитнес-клубом**

Разработайте программное обеспечение для учета и управления фитнес-клубом. Программа должна предоставлять возможность создания записей о клиентах, тренерах, абонементах, тренировках и других данных. Также программа должна поддерживать выполнение различных операций с данными, таких как добавление, редактирование, удаление, поиск и анализ.

In [1]:
from abc import ABC, abstractmethod
from datetime import date
from typing import Optional
from abc import ABCMeta

class MemberMeta(ABCMeta):
    _registry = {}

    def __new__(cls, name, bases, dct):
        new_class = super().__new__(cls, name, bases, dct)
        if "Member" in [base.__name__ for base in bases]:
            cls._registry[name.lower()] = new_class
        return new_class

    @classmethod
    def get_class(cls, name: str):
        return cls._registry.get(name.lower())

    @classmethod
    def list_registered_classes(cls):
        return cls._registry
class Member(ABC, metaclass=MemberMeta):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date):
        self._member_id = member_id
        self._name = name
        self._age = age
        self._membership_type = membership_type
        self._join_date = join_date

    @property
    def member_id(self) -> int:
        return self._member_id

    @property
    def name(self) -> str:
        return self._name

    @name.setter
    def name(self, value: str):
        self._name = value

    @property
    def age(self) -> int:
        return self._age

    @age.setter
    def age(self, value: int):
        if value < 0:
            raise ValueError("Возраст не может быть отрицательным")
        self._age = value

    @property
    def membership_type(self) -> str:
        return self._membership_type

    @membership_type.setter
    def membership_type(self, value: str):
        self._membership_type = value

    @property
    def join_date(self) -> date:
        return self._join_date

    @abstractmethod
    def get_membership_info(self) -> str:
        pass

    def __str__(self) -> str:
        return f"Участник: {self.name}, Тип членства: {self.membership_type}"

    def __lt__(self, other: 'Member') -> bool:
        return self.age < other.age

    def __gt__(self, other: 'Member') -> bool:
        return self.age > other.age

    def __eq__(self, other: 'Member') -> bool:
        return self.member_id == other.member_id

class Client(Member):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date, subscription: str):
        super().__init__(member_id, name, age, membership_type, join_date)
        self._subscription = subscription
    def to_dict(self) -> dict:
        base_data = super().to_dict()
        base_data.update({"subscription": self.subscription})
        return base_data

    @classmethod
    def from_dict(cls, data: dict):
        base_instance = super().from_dict(data)
        return cls(
            member_id=base_instance.member_id,
            name=base_instance.name,
            age=base_instance.age,
            membership_type=base_instance.membership_type,
            join_date=base_instance.join_date,
            subscription=data["subscription"],
        )
    @classmethod
    def log_action(self, action):
        print(f"Log: {action}")

    def send_notification(self, message: str):
        print(f"Уведомление для {self.name}: {message}")
    @property
    def subscription(self) -> str:
        return self._subscription

    @subscription.setter
    def subscription(self, value: str):
        self._subscription = value

    def get_membership_info(self) -> str:
        return f"Клиент: {self.name}, Абонемент: {self.subscription}"

    def __str__(self) -> str:
        return f"Клиент: {self.name}, Абонемент: {self.subscription}"


class Trainer(Member):
    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date, specialization: str):
        super().__init__(member_id, name, age, membership_type, join_date)
        self._specialization = specialization
    def to_dict(self) -> dict:
        base_data = super().to_dict()
        base_data.update({"specialization": self.specialization})
        return base_data

    @classmethod
    def from_dict(cls, data: dict):
        base_instance = super().from_dict(data)
        return cls(
            member_id=base_instance.member_id,
            name=base_instance.name,
            age=base_instance.age,
            membership_type=base_instance.membership_type,
            join_date=base_instance.join_date,
            specialization=data["specialization"],
        )
    @property
    def specialization(self) -> str:
        return self._specialization

    @specialization.setter
    def specialization(self, value: str):
        self._specialization = value

    def get_membership_info(self) -> str:
        return f"Тренер: {self.name}, Специализация: {self.specialization}"

    def __str__(self) -> str:
        return f"Тренер: {self.name}, Специализация: {self.specialization}"


class Location:
    def __init__(self, address: str, room_number: int):
        self.address = address
        self.room_number = room_number

    def __str__(self) -> str:
        return f"Адрес: {self.address}, Комната: {self.room_number}"


class GymClass:
    def __init__(self, class_name: str, trainer: Trainer, schedule: str, location: Location):
        self.class_name = class_name
        self.trainer = trainer
        self.schedule = schedule
        self.location = location
        self.participants = []

    def add_participant(self, participant: Member):
        self.participants.append(participant)

    def remove_participant(self, participant: Member):
        self.participants.remove(participant)

    def get_participants(self) -> list[Member]:
        return self.participants

    def __str__(self) -> str:
        return f"Тренировка: {self.class_name}, Тренер: {self.trainer.name}, Расписание: {self.schedule}"

from abc import ABC, abstractmethod

class Bookable(ABC):
    @abstractmethod
    def book_class(self, member: Member):
        pass


class Reportable(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass


class LoggingMixin:
    def log_action(self, action: str):
        print(f"[LOG] {action}")


class NotificationMixin:
    def send_notification(self, message: str):
        print(f"[NOTIFICATION] {message}")



class MemberFactory:
    @staticmethod
    def create_member(member_type: str, *args, **kwargs) -> Member:
        member_class = MemberMeta.get_class(member_type)
        if not member_class:
            raise ValueError(f"Неизвестный тип участника: {member_type}")
        return member_class(*args, **kwargs)

class Handler:
    def __init__(self, successor=None):
        self.successor = successor

    def handle(self, months: int):
        if self.successor:
            return self.successor.handle(months)
        return False


class Administrator(Handler):
    def handle(self, months: int):
        if months <= 1:
            print(f"Администратор одобрил продление на {months} месяц(ев)")
            return True
        return super().handle(months)


class Manager(Handler):
    def handle(self, months: int):
        if months <= 3:
            print(f"Менеджер одобрил продление на {months} месяц(ев)")
            return True
        return super().handle(months)


class Director(Handler):
    def handle(self, months: int):
        print(f"Директор одобрил продление на {months} месяц(ев)")
        return True

class BookingProcess(ABC):
    def book_class(self, member: Member, gym_class: GymClass):
        if not self.check_availability(gym_class):
            raise ClassFullError("Тренировка переполнена")
        self.add_participant(member, gym_class)
        self.confirm_booking(member, gym_class)

    @abstractmethod
    def check_availability(self, gym_class: GymClass) -> bool:
        pass

    @abstractmethod
    def add_participant(self, member: Member, gym_class: GymClass):
        pass

    @abstractmethod
    def confirm_booking(self, member: Member, gym_class: GymClass):
        pass


class ClientBookingProcess(BookingProcess):
    def check_availability(self, gym_class: GymClass) -> bool:
        return len(gym_class.get_participants()) < 10

    def add_participant(self, member: Member, gym_class: GymClass):
        gym_class.add_participant(member)

    def confirm_booking(self, member: Member, gym_class: GymClass):
        print(f"{member.name} успешно записан на тренировку {gym_class.class_name}")

def check_permissions(permission_level: str):
    def decorator(func):
        def wrapper(user_role: str, *args, **kwargs):
            if user_role not in permission_level:
                raise PermissionDeniedError("Недостаточно прав")
            return func(*args, **kwargs)
        return wrapper
    return decorator

class InvalidMemberError(Exception):
    def __init__(self, message="Некорректные данные участника"):
        super().__init__(message)


class PermissionDeniedError(Exception):
    def __init__(self, message="Недостаточно прав для выполнения действия"):
        super().__init__(message)


class ClassFullError(Exception):
    def __init__(self, message="Тренировка переполнена"):
        super().__init__(message)

import json
from datetime import date

class Member(ABC):
    def to_dict(self) -> dict:
        return {
            "member_id": self.member_id,
            "name": self.name,
            "age": self.age,
            "membership_type": self.membership_type,
            "join_date": self.join_date.isoformat(),
        }

    @classmethod
    def from_dict(cls, data: dict):
        join_date = date.fromisoformat(data["join_date"])
        return cls(
            member_id=data["member_id"],
            name=data["name"],
            age=data["age"],
            membership_type=data["membership_type"],
            join_date=join_date,
        )





import os

def save_to_json(data: list, file_path: str):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump([item.to_dict() for item in data], f, ensure_ascii=False, indent=4)


def load_from_json(file_path: str, cls):
    if not os.path.exists(file_path):
        return []
    with open(file_path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    return [cls.from_dict(item) for item in raw_data]

class Member(ABC):
    # ... (остальной код класса Member)

    def __lt__(self, other: 'Member') -> bool:
        if isinstance(other, Member):
            return self.join_date < other.join_date
        return NotImplemented

    def __gt__(self, other: 'Member') -> bool:
        if isinstance(other, Member):
            return self.join_date > other.join_date
        return NotImplemented

    def __eq__(self, other: 'Member') -> bool:
        if isinstance(other, Member):
            return self.member_id == other.member_id
        return NotImplemented

import logging

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler("gym.log"),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)
    return logger


logger = setup_logging()


class LoggingMixin:
    def log_action(self, action: str):
        logger.info(f"[LOG] {action}")

class Member(ABC):
    """
    Абстрактный класс, представляющий участника фитнес-клуба.
    """

    def __init__(self, member_id: int, name: str, age: int, membership_type: str, join_date: date):
        """
        Инициализирует нового участника.

        :param member_id: Уникальный идентификатор участника.
        :param name: Имя участника.
        :param age: Возраст участника.
        :param membership_type: Тип членства.
        :param join_date: Дата вступления.
        """
        self._member_id = member_id
        self._name = name
        self._age = age
        self._membership_type = membership_type
        self._join_date = join_date


from datetime import date
import logging

# Настройка логирования
logger = setup_logging()


class GymClub:
    """
    Класс для управления фитнес-клубом.
    """
    def __init__(self):
        self.members = []
        self.gym_classes = []

    def add_member(self, member: Member):
        """Добавляет участника в клуб."""
        self.members.append(member)
        logger.info(f"Участник {member.name} добавлен в клуб")

    def remove_member(self, member_id: int):
        """Удаляет участника из клуба по ID."""
        for member in self.members:
            if member.member_id == member_id:
                self.members.remove(member)
                logger.info(f"Участник {member.name} удален из клуба")
                return
        raise InvalidMemberError(f"Участник с ID {member_id} не найден")

    def get_all_members(self) -> list[Member]:
        """Возвращает список всех участников клуба."""
        return self.members

    def search_by_name(self, name: str) -> list[Member]:
        """Ищет участников по имени."""
        return [member for member in self.members if name.lower() in member.name.lower()]

    def add_gym_class(self, gym_class: GymClass):
        """Добавляет тренировку в клуб."""
        self.gym_classes.append(gym_class)
        logger.info(f"Тренировка {gym_class.class_name} добавлена в клуб")

    def book_class(self, member: Member, gym_class: GymClass):
        """Записывает участника на тренировку."""
        try:
            booking_process = ClientBookingProcess()
            booking_process.book_class(member, gym_class)
            member.log_action(f"{member.name} записан на тренировку {gym_class.class_name}")
        except ClassFullError as e:
            logger.error(str(e))
            member.send_notification(f"Не удалось записаться на тренировку {gym_class.class_name}: {e}")

# Инициализация фабрики участников
class MemberFactory:
    @staticmethod
    def create_member(member_type: str, *args, **kwargs) -> Member:
        member_class = MemberMeta.get_class(member_type)
        if not member_class:
            raise ValueError(f"Неизвестный тип участника: {member_type}")
        return member_class(*args, **kwargs)


# Создание фитнес-клуба
club = GymClub()

# 1. Создание участников
client1 = MemberFactory.create_member("client", 1, "Alice", 30, "Basic", date.today(), "Monthly")
client2 = MemberFactory.create_member("client", 2, "Vlad", 24, "Basic", date.today(), "Monthly")
trainer1 = MemberFactory.create_member("trainer", 3, "Bob", 40, "Premium", date.today(), "Yoga")

# Добавление участников в клуб
club.add_member(client1)
club.add_member(client2)
club.add_member(trainer1)

# 2. Управление данными
# Редактирование данных участников
client1.name = "Alice Smith"
client1.age = 31
logger.info(f"Данные участника {client1.name} обновлены")
client2.name = "Vlad Redford"
client2.age = 25
logger.info(f"Данные участника {client2.name} обновлены")

# Удаление участника
club.remove_member(2)

# 3. Анализ данных
# Получение списка всех участников
all_members = club.get_all_members()
print("Все участники клуба:")
for member in all_members:
    print(member)

# Поиск участников по имени
search_results = club.search_by_name("Alice")
print("\nРезультаты поиска по имени 'Alice':")
for member in search_results:
    print(member)

# 4. Логирование и уведомления
# Логирование действий
client1.log_action("Клиент Alice обновил свой профиль")
client1.send_notification("Ваш абонемент истекает через 5 дней")

# 5. Обработка исключений
# Создание тренировки
location = Location("Main Street", 101)
gym_class = GymClass("Yoga Class", trainer1, "10:00 AM", location)
club.add_gym_class(gym_class)

# Запись участников на тренировку
try:
    club.book_class(client1, gym_class)
except ClassFullError as e:
    logger.error(str(e))

# Проверка прав доступа
@check_permissions("admin")
def admin_action():
    print("Административное действие выполнено")

try:
    admin_action(user_role="user")  # Пользователь без прав
except PermissionDeniedError as e:
    logger.error(str(e))

ERROR:__main__:Недостаточно прав


Все участники клуба:
Клиент: Alice Smith, Абонемент: Monthly
Тренер: Bob, Специализация: Yoga

Результаты поиска по имени 'Alice':
Клиент: Alice Smith, Абонемент: Monthly
Log: Клиент Alice обновил свой профиль
Уведомление для Alice Smith: Ваш абонемент истекает через 5 дней
Alice Smith успешно записан на тренировку Yoga Class
Log: Alice Smith записан на тренировку Yoga Class
